<a href="https://colab.research.google.com/github/rohhj622/windPowerPrediction/blob/master/code1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from keras import models, layers, Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Input, SimpleRNN

## 1) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 3) 데이터 불러오기 시작

In [ ]:
allData = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/preprocessing_Data.csv') # 컬럼명 숫자로 들어가게
allData = allData.drop('Unnamed: 0',1)
allData

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem
0,2013-01-01 00:00:00,2013-01-01 00:00:00,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0
1,2013-01-01 01:00:00,2013-01-01 00:00:00,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0
2,2013-01-01 02:00:00,2013-01-01 00:00:00,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0
3,2013-01-01 03:00:00,2013-01-01 00:00:00,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3
4,2013-01-01 04:00:00,2013-01-01 00:00:00,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,2018-12-31 19:00:00,2018-12-31 00:00:00,19:00,8699,4.3,4.7,340,58.0,4.8,-3.2,1032.1,1034.7,3.3
52580,2018-12-31 20:00:00,2018-12-31 00:00:00,20:00,10694,3.9,4.5,360,61.0,4.9,-2.9,1032.0,1034.6,2.9
52581,2018-12-31 21:00:00,2018-12-31 00:00:00,21:00,5494,3.5,4.0,360,59.0,4.6,-3.7,1031.9,1034.5,1.9
52582,2018-12-31 22:00:00,2018-12-31 00:00:00,22:00,2718,3.8,3.4,20,61.0,4.9,-3.0,1031.6,1034.2,2.1


# 2. 데이터 가공하기 

## 1) datetime 컬럼 제작, datetime 형식으로 변환

In [ ]:
df = allData.copy()
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M', errors='ignore') #합친거 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d', errors='ignore')  #date형식으로 변환
#df['time'] = pd.to_datetime(df['time'],format='%H:%M', errors='ignore') #time 형식으로 변환
df.head(5)

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1


## 2) 년(year), 월(month), 일(day), 계절(season) 컬럼 생성



In [ ]:
df['year'] = df['date'].dt.year # 년
df['month'] = df['date'].dt.month # 월
df['day'] = df['date'].dt.day# 일

conditionlist = [
    (df['month'] == 3)|(df['month'] == 4)| (df['month'] == 5) ,
    (df['month'] == 6)|(df['month'] == 7)|(df['month'] == 8) ,
    (df['month'] == 9)|(df['month'] == 10)|(df['month'] == 11),
    (df['month'] == 12)|(df['month'] == 1)|(df['month'] == 2) ]
choicelist = ['0', '1', '2','3']
#choicelist = ['spring', 'summer', 'autumn','winter']
df['season'] = np.select(conditionlist, choicelist, default='Not Specified')
df.head(5)

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem,year,month,day,season
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0,2013,1,1,3
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0,2013,1,1,3
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0,2013,1,1,3
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3,2013,1,1,3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1,2013,1,1,3


# 4. 훈련 & 예측

## 1) train-test data 나누기

In [ ]:
df2 = df.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          52584 non-null  datetime64[ns]
 1   date              52584 non-null  datetime64[ns]
 2   time              52584 non-null  object        
 3   windForce         52584 non-null  int64         
 4   tem               52584 non-null  float64       
 5   windSpeed         52584 non-null  float64       
 6   windDirect        52584 non-null  int64         
 7   humidity          52584 non-null  float64       
 8   vaporPressure     52584 non-null  float64       
 9   dewPoint          52584 non-null  float64       
 10  pressure          52584 non-null  float64       
 11  seaLevelPressure  52584 non-null  float64       
 12  groundTem         52584 non-null  float64       
 13  year              52584 non-null  int64         
 14  month             5258

In [ ]:
X = df2[['tem','windSpeed','windDirect','humidity']] #,'groundTem','seaLevelPressure'
y = df2[['windForce']]

transform = MinMaxScaler()
trans_X=transform.fit_transform(X)
trans_y=transform.fit_transform(y)

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)
print('train data 개수 : ', len(X_train))
print('test data 개수: ', len(X_test))
print(len(X_train)+len(X_test))

train data 개수 :  42067
test data 개수:  10517
52584


In [ ]:
X_train.ndim
X_train.shape

(42067, 4)

## 5. MakeDataSet


In [ ]:
# https://teddylee777.github.io/tensorflow/LSTM%EC%9C%BC%EB%A1%9C-%EC%98%88%EC%B8%A1%ED%95%B4%EB%B3%B4%EB%8A%94-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%A3%BC%EA%B0%80
def make_dataset(data, label, window_size=1):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
# X = df2[['tem','windSpeed','humidity','groundTem','windDirect','seaLevelPressure']] #,'windDirect'
# X = df2[['tem','windSpeed','groundTem']] #,'windDirect'
transform = MinMaxScaler()

scale_cols=['tem','windSpeed','groundTem','windForce']
df_scaled = df2[['tem','windSpeed','groundTem','windForce']] 
df_scaled = transform.fit_transform(df_scaled)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

train = df_scaled[:int(len(df2)*0.8)]
test = df_scaled[int(len(df2)*0.8):]
print(train.head())
print(test.head())

        tem  windSpeed  groundTem  windForce
0  0.190123   0.180791   0.061135   0.000000
1  0.180247   0.146893   0.061135   0.000000
2  0.195062   0.169492   0.061135   0.007663
3  0.224691   0.096045   0.065502   0.022407
4  0.172840   0.101695   0.062591   0.045065
            tem  windSpeed  groundTem  windForce
42067  0.607407   0.158192   0.312955   0.144940
42068  0.585185   0.062147   0.305677   0.111370
42069  0.565432   0.067797   0.298399   0.079967
42070  0.558025   0.073446   0.292576   0.061808
42071  0.562963   0.079096   0.289665   0.055560


In [ ]:
WINDOW_SIZE = 1 # 일 데이터를 묶을지 (window_size : 20, 1)
feature1 = ['tem','windSpeed','groundTem','windForce']
labels1 = ['windForce']

train_feature = train[feature1]
train_label = train[labels1]

train_feature, train_label = make_dataset(train_feature, train_label, WINDOW_SIZE)

X_train, X_valid,y_train, y_valid = train_test_split(train_feature,train_label,test_size=0.2,random_state=10) # train, valid
print(X_train.shape, X_valid.shape)

(33652, 1, 4) (8414, 1, 4)


In [ ]:
test_feature = test[feature1]
test_label = test[labels1]
test_feature, test_label = make_dataset(test_feature, test_label, WINDOW_SIZE)
test_feature.shape, test_label.shape

((10516, 1, 4), (10516, 1))

### model loss function

In [ ]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test1, y_predict):
    return np.sqrt(mean_squared_error(y_test1, y_predict))

def MSE(y_test1, y_predict):
    return mean_squared_error(y_test1, y_predict)

In [ ]:
def make2D(arr1):
  res_arr = []
  for i in range(0,len(arr1)):
    res_arr.append(arr1[i][0])
  return res_arr

### nn model

In [ ]:
# 반복 횟수
num1 = 10

# model 저장 위치
model_path = '/content/gdrive/My Drive/Colab Notebooks/1-6-model'

mse_arr = [0,0,0,0,0]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
import os

In [ ]:
nn_model = Sequential()
nn_model.add(Dense(128, input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu'))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))

In [ ]:
nn_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
#early_stop = EarlyStopping(monitor='val_loss', patience=5)
#filename = os.path.join(model_path, 'tmp_nn_model2.h5')
#checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
nn_arr=[]
j = 0
for i in range(10,105,10):
  nn_model = Sequential()
  nn_model.add(Dense(128, input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu'))
  nn_model.add(Dense(64, activation='relu'))
  nn_model.add(Dense(32, activation='relu'))
  nn_model.add(Dense(16, activation='relu'))
  nn_model.add(Dense(1, activation='sigmoid'))
  nn_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
  nn_model.fit(X_train, y_train, epochs=i, batch_size=16)
  nn_pred = nn_model.predict(test_feature)
  nn_arr.append(MSE(test_label, make2D(nn_pred)))
  j = j + 1


Epoch 1/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0166 - mae: 0.0861 - acc: 0.1223
Epoch 2/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0094 - mae: 0.0631 - acc: 0.1224
Epoch 3/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0092 - mae: 0.0623 - acc: 0.1226
Epoch 4/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0091 - mae: 0.0613 - acc: 0.1227
Epoch 5/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0090 - mae: 0.0611 - acc: 0.1202
Epoch 6/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0090 - mae: 0.0607 - acc: 0.1215
Epoch 7/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0089 - mae: 0.0605 - acc: 0.1216
Epoch 8/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0087 - mae: 0.0600 - acc: 0.1211
Epoch 9/10
2104/2104 [==============================] - 2s 1ms/step - loss: 0.0087 - mae: 0.0598 - acc: 0.1219
E

In [ ]:
nn_arr

[0.00970020131381706,
 0.009335357292147262,
 0.009219378154053373,
 0.009616747920205579,
 0.009204452141425425,
 0.009271725641016573,
 0.009206723022005219,
 0.009141134746412745,
 0.009200005615334537,
 0.00921309169856088]

In [ ]:
# nn_arr=[]
# for i in range(0,num1):
#   hist1 = nn_model.fit(X_train, y_train, 
#                     epochs=200, 
#                     batch_size=16,
#                     validation_data=(X_valid, y_valid), 
#                     callbacks=[early_stop, checkpoint])
#   # weight 로딩
#   nn_model.load_weights(filename)

#   # 예측
#   nn_pred = nn_model.predict(test_feature)

#   mse_arr[0] = mse_arr[0] + MSE(test_label, make2D(nn_pred))
  
#   nn_arr.append(MSE(test_label, make2D(nn_pred)))
 

In [ ]:
# nn_arr=[]
# for i in range(0,num1):
#   hist1 = nn_model.fit(X_train, y_train, 
#                     epochs=200, 
#                     batch_size=16,
#                     validation_data=(X_valid, y_valid), 
#                     callbacks=[early_stop, checkpoint])
#   # weight 로딩
#   nn_model.load_weights(filename)

#   # 예측
#   nn_pred = nn_model.predict(test_feature)

#   mse_arr[0] = mse_arr[0] + MSE(test_label, make2D(nn_pred))
  
#   nn_arr.append(MSE(test_label, make2D(nn_pred)))
 

In [ ]:
nn_arr

[0.00970020131381706,
 0.009335357292147262,
 0.009219378154053373,
 0.009616747920205579,
 0.009204452141425425,
 0.009271725641016573,
 0.009206723022005219,
 0.009141134746412745,
 0.009200005615334537,
 0.00921309169856088]

### lstm model

In [ ]:
# lstm_model = Sequential()
# lstm_model.add(LSTM(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
# lstm_model.add(Dense(64, activation='relu'))
# lstm_model.add(Dense(32, activation='relu'))
# lstm_model.add(Dense(16, activation='relu'))
# lstm_model.add(Dense(1, activation='sigmoid'))

# # network.add(layers.Dense(128, input_shape=train_feature.shape,activation='relu'))
# # network.add(layers.Dense(64,activation='relu'))
# # network.add(layers.Dense(1))

In [ ]:
lstm_arr=[]
j = 0
for i in range(10,105,10):
  lstm_model = Sequential()
  lstm_model.add(LSTM(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
  lstm_model.add(Dense(64, activation='relu'))
  lstm_model.add(Dense(32, activation='relu'))
  lstm_model.add(Dense(16, activation='relu'))
  lstm_model.add(Dense(1, activation='sigmoid'))
  lstm_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
  lstm_model.fit(X_train, y_train, epochs=i, batch_size=16)
  pred = lstm_model.predict(test_feature)
  lstm_arr.append(MSE(test_label, pred))
  j = j + 1


In [ ]:
lstm_arr

[0.009192938330442069,
 0.009243093685607485,
 0.009274832137943455,
 0.009157417664020992,
 0.009150139312317401,
 0.009195315595668373,
 0.009216292171638642,
 0.009282204723494739,
 0.009135343135507611,
 0.009374412335835875]

In [ ]:
# lstm_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = os.path.join(model_path, 'tmp_lstm2.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
# for i in range(0,num1):
#   hist2 = lstm_model.fit(X_train, y_train, 
#                     epochs=200, 
#                     batch_size=16,
#                     validation_data=(X_valid, y_valid), 
#                     callbacks=[early_stop, checkpoint])
#   # weight 로딩
#   lstm_model.load_weights(filename)

#   # 예측
#   pred = lstm_model.predict(test_feature)
  
#   mse_arr[1] = mse_arr[1] + MSE(test_label, pred)

In [ ]:
# print(MSE(test_label, pred)) 
# print(RMSE(test_label, pred))
# print(lstm_model.evaluate(X_train, y_train))

### RNN

In [ ]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dense(16, activation='relu'))
rnn_model.add(Dense(1, activation='sigmoid'))

# network.add(layers.Dense(128, input_shape=train_feature.shape,activation='relu'))
# network.add(layers.Dense(64,activation='relu'))
# network.add(layers.Dense(1))


In [ ]:
rnn_arr=[]
j = 0
for i in range(10,105,10):
  rnn_model = Sequential()
  rnn_model.add(SimpleRNN(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
  rnn_model.add(Dense(64, activation='relu'))
  rnn_model.add(Dense(32, activation='relu'))
  rnn_model.add(Dense(16, activation='relu'))
  rnn_model.add(Dense(1, activation='sigmoid'))
  rnn_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
  rnn_model.fit(X_train, y_train, epochs=i, batch_size=16)
  pred = rnn_model.predict(test_feature)
  rnn_arr.append(MSE(test_label, pred))
  j = j + 1


Epoch 1/10
2104/2104 [==============================] - 4s 1ms/step - loss: 0.0184 - mae: 0.0904 - acc: 0.1216
Epoch 2/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0090 - mae: 0.0629 - acc: 0.1248
Epoch 3/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0090 - mae: 0.0617 - acc: 0.1180
Epoch 4/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0089 - mae: 0.0615 - acc: 0.1210
Epoch 5/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0089 - mae: 0.0610 - acc: 0.1180
Epoch 6/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0091 - mae: 0.0613 - acc: 0.1195
Epoch 7/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0090 - mae: 0.0611 - acc: 0.1212
Epoch 8/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0093 - mae: 0.0618 - acc: 0.1202
Epoch 9/10
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0087 - mae: 0.0595 - acc: 0.1200
E

In [ ]:
rnn_arr

[0.009192558647222118,
 0.00947279195016218,
 0.009215553070711821,
 0.009304933609719167,
 0.009172418332127398,
 0.0093012471841463,
 0.009245733523868515,
 0.00921720550163844,
 0.00911843791081037,
 0.009213675604421778]

In [ ]:
# rnn_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = os.path.join(model_path, 'tmp_rnn2.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
# for i in range(0,num1):
#   hist3 = rnn_model.fit(X_train, y_train, 
#                     epochs=200, 
#                     batch_size=16,
#                     validation_data=(X_valid, y_valid), 
#                     callbacks=[early_stop, checkpoint])
#   # weight 로딩
#   rnn_model.load_weights(filename)

#   # 예측
#   rnn_pred = rnn_model.predict(test_feature)
#   mse_arr[2] = mse_arr[2] + MSE(test_label, rnn_pred)

In [ ]:
# print(MSE(test_label, rnn_pred)) 
# print(RMSE(test_label, rnn_pred))
# print(rnn_model.evaluate(X_train, y_train))

### RNN + LSTM

In [ ]:
# rnn_lstm_model = Sequential()
# rnn_lstm_model.add(SimpleRNN(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
# rnn_lstm_model.add(LSTM(64,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
# rnn_lstm_model.add(Dense(32, activation='relu'))
# rnn_lstm_model.add(Dense(16, activation='relu'))
# rnn_lstm_model.add(Dense(1, activation='sigmoid'))

In [ ]:
rnn_lstm_arr=[]
j = 0
for i in range(10,105,10):
  rnn_lstm_model = Sequential()
  rnn_lstm_model.add(SimpleRNN(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
  rnn_lstm_model.add(LSTM(64,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
  rnn_lstm_model.add(Dense(32, activation='relu'))
  rnn_lstm_model.add(Dense(16, activation='relu'))
  rnn_lstm_model.add(Dense(1, activation='sigmoid'))
  rnn_lstm_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
  rnn_lstm_model.fit(X_train, y_train, epochs=i, batch_size=16)
  pred = rnn_lstm_model.predict(test_feature)
  rnn_lstm_arr.append(MSE(test_label, pred))
  j = j + 1


Epoch 1/10
2104/2104 [==============================] - 6s 2ms/step - loss: 0.0259 - mae: 0.1075 - acc: 0.1207
Epoch 2/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0093 - mae: 0.0633 - acc: 0.1208
Epoch 3/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0093 - mae: 0.0622 - acc: 0.1182
Epoch 4/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0092 - mae: 0.0623 - acc: 0.1193
Epoch 5/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0092 - mae: 0.0613 - acc: 0.1231
Epoch 6/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0091 - mae: 0.0614 - acc: 0.1196
Epoch 7/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0089 - mae: 0.0602 - acc: 0.1231
Epoch 8/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0091 - mae: 0.0608 - acc: 0.1214
Epoch 9/10
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0087 - mae: 0.0598 - acc: 0.1239
E

In [ ]:
rnn_lstm_arr

[0.00920649681419307,
 0.009184841698602361,
 0.009248753799325363,
 0.009208356522338642,
 0.009188167692304012,
 0.009221370733853667,
 0.009190359816388109,
 0.009196405679560274,
 0.009121120563884503,
 0.0092259575204397]

In [ ]:
# rnn_lstm_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = os.path.join(model_path, 'tmp_rnn_lstm2.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
# for i in range(0,num1):
#   hist4 = rnn_lstm_model.fit(X_train, y_train, 
#                     epochs=200, 
#                     batch_size=16,
#                     validation_data=(X_valid, y_valid), 
#                     callbacks=[early_stop, checkpoint])
#   # weight 로딩
#   rnn_lstm_model.load_weights(filename)

#   # 예측
#   rnn_lstm_pred = rnn_lstm_model.predict(test_feature)

#   mse_arr[3] = mse_arr[3] + MSE(test_label, rnn_lstm_pred)

In [ ]:
# print(MSE(test_label, rnn_lstm_pred)) 
# print(RMSE(test_label, rnn_lstm_pred))
# print(rnn_model.evaluate(X_train, y_train))

### RNN + LSTM1

In [ ]:
# rnn_lstm1_arr=[]
# j = 0
# for i in range(10,105,10):
#   rnn_lstm1_model = Sequential()
#   rnn_lstm1_model.add(LSTM(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
#   rnn_lstm1_model.add(SimpleRNN(64,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
#   rnn_lstm1_model.add(LSTM(32,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
#   rnn_lstm1_model.add(LSTM(16,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
#   rnn_lstm1_model.add(Dense(1, activation='sigmoid'))
#   rnn_lstm1_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
#   rnn_lstm1_model.fit(X_train, y_train, epochs=i, batch_size=16)
#   pred = rnn_lstm1_model.predict(test_feature)
#   rnn_lstm1_arr.append(MSE(test_label, pred))
#   j = j + 1

In [ ]:
rnn_lstm1_arr=[]
j = 0
for i in range(10,105,10):
  rnn_lstm1_model = Sequential()
  rnn_lstm1_model.add(LSTM(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
  rnn_lstm1_model.add(SimpleRNN(64,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
  rnn_lstm1_model.add(LSTM(32,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
  rnn_lstm1_model.add(SimpleRNN(16,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
  rnn_lstm1_model.add(Dense(1, activation='sigmoid'))
  rnn_lstm1_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
  rnn_lstm1_model.fit(X_train, y_train, epochs=i, batch_size=16)
  pred = rnn_lstm1_model.predict(test_feature)
  rnn_lstm1_arr.append(MSE(test_label, pred))
  j = j + 1

Epoch 1/10
2104/2104 [==============================] - 10s 3ms/step - loss: 0.0282 - mae: 0.1124 - acc: 0.1209
Epoch 2/10
2104/2104 [==============================] - 7s 3ms/step - loss: 0.0091 - mae: 0.0627 - acc: 0.1185
Epoch 3/10
2104/2104 [==============================] - 7s 3ms/step - loss: 0.0091 - mae: 0.0615 - acc: 0.1237
Epoch 4/10
2104/2104 [==============================] - 6s 3ms/step - loss: 0.0091 - mae: 0.0617 - acc: 0.1178
Epoch 5/10
2104/2104 [==============================] - 7s 3ms/step - loss: 0.0091 - mae: 0.0614 - acc: 0.1200
Epoch 6/10
2104/2104 [==============================] - 7s 3ms/step - loss: 0.0089 - mae: 0.0611 - acc: 0.1174
Epoch 7/10
2104/2104 [==============================] - 7s 3ms/step - loss: 0.0089 - mae: 0.0603 - acc: 0.1207
Epoch 8/10
2104/2104 [==============================] - 6s 3ms/step - loss: 0.0087 - mae: 0.0597 - acc: 0.1252
Epoch 9/10
2104/2104 [==============================] - 7s 3ms/step - loss: 0.0089 - mae: 0.0606 - acc: 0.1196


In [ ]:
rnn_lstm1_arr

[0.009377103789916708,
 0.009454277207221796,
 0.009182171428655284,
 0.00930766932591977,
 0.00935383019570444,
 0.009215125758704281,
 0.00918861202284939,
 0.00917370226587434,
 0.009237837914881369,
 0.009256684012205715]

In [ ]:
# rnn_lstm1_model = Sequential()
# rnn_lstm1_model.add(LSTM(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
# rnn_lstm1_model.add(SimpleRNN(64,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
# rnn_lstm1_model.add(LSTM(32,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=True))
# rnn_lstm1_model.add(LSTM(16,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
# rnn_lstm1_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# rnn_lstm1_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = os.path.join(model_path, 'tmp_rnn_lstm3.h5')
# print(filename)
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

/content/gdrive/My Drive/Colab Notebooks/1-6-model/tmp_rnn_lstm3.h5


In [ ]:
# for i in range(0,num1):
#   hist5 = rnn_lstm1_model.fit(X_train, y_train, 
#                     epochs=200, 
#                     batch_size=16,
#                     validation_data=(X_valid, y_valid), 
#                     callbacks=[early_stop, checkpoint])
#   # weight 로딩
#   rnn_lstm1_model.load_weights(filename)

#   # 예측
#   rnn_lstm1_pred = rnn_lstm1_model.predict(test_feature)
#   mse_arr[4] = mse_arr[4] + MSE(test_label, rnn_lstm1_pred)

2104/2104 [==============================] - 8s 4ms/step - loss: 0.0088 - mae: 0.0598 - acc: 0.1212 - val_loss: 0.0090 - val_mae: 0.0588 - val_acc: 0.1192

Epoch 00005: val_loss did not improve from 0.00874
Epoch 6/200
  15/2104 [..............................] - ETA: 7s - loss: 0.0091 - mae: 0.0532 - acc: 0.1125 

KeyboardInterrupt: ignored

In [ ]:
fig, loss_ax = plt.subplots(figsize=(16,9))
acc_ax = loss_ax.twinx()
plt.title('RNN_LSTM2')
loss_ax.plot(hist5.history['loss'], 'y', label='train loss')
loss_ax.plot(hist5.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist5.history['acc'], 'b', label='train acc')
acc_ax.plot(hist5.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper right')

In [ ]:
# ## 
# print('----------------NN-----------------')
# nn_mse = MSE(test_label, make2D(nn_pred))
# print(MSE(test_label, make2D(nn_pred)))
# print(RMSE(test_label, pred))
# print(nn_model.evaluate(X_train, y_train))
# print('\n----------------LSTM-----------------')
# lstm_mse = MSE(test_label, pred)
# print(MSE(test_label, pred)) 
# print(RMSE(test_label, pred))
# print(lstm_model.evaluate(X_train, y_train))
# print('\n----------------RNN-----------------')
# rnn_mse = MSE(test_label, rnn_pred)
# print(MSE(test_label, rnn_pred)) 
# print(RMSE(test_label, rnn_pred))
# print(rnn_model.evaluate(X_train, y_train))
# print('\n----------------RNN+LSTM-----------------')
# rnn_lstm1_mse = MSE(test_label, rnn_lstm_pred)
# print(MSE(test_label, rnn_lstm_pred)) 
# print(RMSE(test_label, rnn_lstm_pred))
# print(rnn_model.evaluate(X_train, y_train))
# print('\n----------------RNN+LSTM2-----------------')
# rnn_lstm2_mse = MSE(test_label, rnn_lstm1_pred)
# print(MSE(test_label, rnn_lstm1_pred)) 
# print(RMSE(test_label, rnn_lstm1_pred))
# print(rnn_model.evaluate(X_train, y_train))

In [ ]:
# objects = ('NN', 'LSTM', 'RNN', 'RNN+LSTM', 'RNN+LSTM2')
# y_pos = np.arange(len(objects))
# performance = [nn_mse,lstm_mse,rnn_mse,rnn_lstm1_mse,rnn_lstm2_mse]

# plt.figure(figsize=(16,10))
# plt.plot(y_pos, performance)
# plt.xticks(y_pos, objects)
# plt.xlabel('Algorithm')
# plt.ylabel('MSE')

# plt.show()

In [ ]:
performance = [0,0,0,0,0]
for i in range(0,5):
  performance[i] = mse_arr[i] / 10

In [ ]:
objects = ('NN', 'LSTM', 'RNN', 'RNN+LSTM', 'RNN+LSTM2')
y_pos = np.arange(len(objects))


plt.figure(figsize=(16,10))
plt.plot(y_pos, performance)
plt.xticks(y_pos, objects)
plt.xlabel('Algorithm')
plt.ylabel('MSE')

plt.show()

In [ ]:
performance